In [7]:
from pathlib import Path 
import os, dotenv
dotenv.load_dotenv()
os.chdir(Path(os.getenv("PYTHONPATH")).expanduser())

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [9]:
VARIATION = "nocomments"
jsonl_path = f"data/training_datasets/verified_{VARIATION}.jsonl"
#jsonl_path = "data/training_datasets/verified_nocomments.jsonl"

df = pd.read_json(jsonl_path, lines=True)

In [10]:
df.head()

,filename,filetype,content,variant
0,cryptol/examples/splitAt.cry,cry,"x = [1,2,3,4] : [_][8]\n\ny = (splitAt x) : ([...",without_comments
1,cryptol/examples/AE.cry,cry,module AE where\n\nparameter\n type A : * ...,without_comments
2,cryptol/examples/xor_cipher.cry,cry,encrypt : {a}(fin a) => [8] -> [a][8] -> [a][8...,without_comments
3,cryptol/examples/zero_weird.cry,cry,x : {a}() => a -> [16]\nx v = zero v \n\nprope...,without_comments
4,cryptol/examples/builtin_lifting.cry,cry,"x = [True,False]\ny = [False,True]\n\nproperty...",without_comments


In [11]:
from src.preprocessing.interpreter_process import verify_df_row_with_cryptol
from src.preprocessing.saw_subprocess import run_saw_script, load_saw_results

saw_results = []
MOUNT_DIR = "/Users/josh/SecurityAnalytics/DataPreprocess/cryptol-files"
SERVER_URL = "http://localhost:8080"
#print(verify_df_row_with_cryptol(df, 35, host_mount_dir=MOUNT_DIR, server_url=SERVER_URL))
rows = []
results_df = load_saw_results("data/training_datasets/syntax_check/saw_results.jsonl")
for i in range(len(df)):
    if i % 10 == 0 and i > 0:
        print(f"Processed {i} of {len(df)} files")
    if df.iloc[i]["filetype"] == 'cry':
        repl_result = verify_df_row_with_cryptol(
                df, i, host_mount_dir=MOUNT_DIR, server_url=SERVER_URL)
        rows.append({"filename" : df.iloc[i]["filename"], **repl_result["load_info"]})
    else:
        if df.iloc[i]["filename"] in results_df['filename'].values:
            result_ = results_df[results_df['filename'] == df.iloc[i]["filename"]]
            rows.append({"filename" : df.iloc[i]["filename"], "load_ok": result_["load_ok"].values[0], "error": result_["error"].values[0] if not result_["load_ok"].values[0] else None, "file_deps": ["SAW"]})
        else:
            saw_result = run_saw_script(f"/Users/josh/SecurityAnalytics/{df.iloc[i]['filename']}")
            row = {"filename": df.iloc[i]["filename"], **saw_result}
            results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)
            #saw_results.append({"filename": df.iloc[i]["filename"], **saw_result})
            if saw_result["returncode"] == 0:
                rows.append({"filename" : df.iloc[i]["filename"], "load_ok": True, "error": None, "file_deps": ["SAW"]})
            else:
                rows.append({"filename" : df.iloc[i]["filename"], "load_ok": False, "error": "saw", "file_deps": ["SAW"]})

Processed 10 of 1312 files
Processed 20 of 1312 files
Processed 30 of 1312 files
Processed 40 of 1312 files
Processed 50 of 1312 files
Processed 60 of 1312 files
Processed 70 of 1312 files
Processed 80 of 1312 files
Processed 90 of 1312 files
Processed 100 of 1312 files
Processed 110 of 1312 files
Processed 120 of 1312 files
Processed 130 of 1312 files
Processed 140 of 1312 files
Processed 150 of 1312 files
Processed 160 of 1312 files
Processed 170 of 1312 files
Processed 180 of 1312 files
Processed 190 of 1312 files
Processed 200 of 1312 files
Processed 210 of 1312 files
Processed 220 of 1312 files
Processed 230 of 1312 files
Processed 240 of 1312 files
Processed 250 of 1312 files
Processed 260 of 1312 files
Processed 270 of 1312 files
Processed 280 of 1312 files
Processed 290 of 1312 files
Processed 300 of 1312 files
Processed 310 of 1312 files
Processed 320 of 1312 files
Processed 330 of 1312 files
Processed 340 of 1312 files
Processed 350 of 1312 files
Processed 360 of 1312 files
P

In [12]:
results_df.head()

,filename,load_ok,returncode,stdout,stderr,cmd,cwd,error
0,cryptol-specs/Primitive/Symmetric/Cipher/Block...,True,0,"[01:25:15.332] Loading file ""/Users/josh/Secur...",,"[saw, /Users/josh/SecurityAnalytics/developmen...",/Users/josh/SecurityAnalytics/development/cryp...,NaN
1,saw-script/intTests/test1533/test.saw,True,0,"[01:25:21.098] Loading file ""/Users/josh/Secur...",,"[saw, /Users/josh/SecurityAnalytics/developmen...",/Users/josh/SecurityAnalytics/development/saw-...,NaN
2,saw-script/intTests/test_property_as_rewrite_r...,True,0,"[01:25:21.596] Loading file ""/Users/josh/Secur...",,"[saw, /Users/josh/SecurityAnalytics/developmen...",/Users/josh/SecurityAnalytics/development/saw-...,NaN
3,saw-script/intTests/test2148/test.saw,True,0,"[01:25:22.022] Loading file ""/Users/josh/Secur...",,"[saw, /Users/josh/SecurityAnalytics/developmen...",/Users/josh/SecurityAnalytics/development/saw-...,NaN
4,saw-script/intTests/test_issue642/test.saw,True,0,"[01:25:23.246] Loading file ""/Users/josh/Secur...",,"[saw, /Users/josh/SecurityAnalytics/developmen...",/Users/josh/SecurityAnalytics/development/saw-...,NaN


In [13]:
repl_results_df = pd.DataFrame(rows)
repl_results_df = repl_results_df.join(df[["filename", "filetype"]].set_index('filename'), on='filename', how='left', rsuffix='_orig')
repl_results_df.head()

,filename,load_ok,file,error,file_deps,filetype
0,cryptol/examples/splitAt.cry,True,files/splitAt_mjlysdk2.cry,None,[Cryptol],cry
1,cryptol/examples/AE.cry,True,files/AE_286_drcm.cry,None,error,cry
2,cryptol/examples/xor_cipher.cry,True,files/xor_cipher_6llbb_4a.cry,None,[Cryptol],cry
3,cryptol/examples/zero_weird.cry,True,files/zero_weird_7pd9vaz7.cry,None,[Cryptol],cry
4,cryptol/examples/builtin_lifting.cry,True,files/builtin_lifting_bwyxzx4p.cry,None,[Cryptol],cry


In [14]:
print(len(repl_results_df[(repl_results_df["load_ok"] == True) & (repl_results_df["filetype"] == "cry")]))
print(len(repl_results_df[(repl_results_df["load_ok"] == False) & (repl_results_df["filetype"] == "cry")]))

665
0


In [15]:
print(len(repl_results_df[(repl_results_df["load_ok"] == True) & (repl_results_df["filetype"] != "cry")]))
print(len(repl_results_df[(repl_results_df["load_ok"] == False) & (repl_results_df["filetype"] != "cry")]))

274
373


In [16]:
print(len(repl_results_df[repl_results_df["load_ok"] == True]))
print(len(repl_results_df[repl_results_df["load_ok"] == False]))

939
373
